In [3]:
import pandas as pd
from datetime import datetime

In [4]:
df = pd.read_csv("data/df_hack_final.csv")

#### Формат имен признаков:
- Ni_rec – извлечение никеля в готовый никелевый продукт, концентрат (значение может отсутствовать, валидны только меньше 1 и больше 0),
- Ore, oreth – имена, которые означают признаки рудного сгустителя (Ore thickener) и на входе первой ФМ (напомним, ФМ – флотомашина),
- resth – имена, которые означают признаки сгустителя с готовым никелевым продуктом (Final Ni thickener).
- Физические характеристики: Mass (масса), Dens (плотность), Vol (объём), Ni (никель), Cu (медь), AU (флаг автоуправления, если оптимизатор управления ФМ включен, то равен 1 – фактические диапазоны на этой ФМ актуальны, в противном случае диапазонам не следует доверять, так как не обновлялись после выключения оптимизатора)
- Суффиксы имён ФМ (положения в цепи агрегатов): 1.1, 1.2, …, 6.2 (см Схему флотации).
- Суффиксы продуктов: F – питание ФМ, C – концентрат ФМ, T – хвосты ФМ.
- Суффиксы границ фактических диапазонов, которые были выставлены технологом для оптимизатора ФМ: min, max.


In [5]:
df = df.astype({'MEAS_DT': 'datetime64[ns]'})

In [6]:
df.describe()

,MEAS_DT,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Mass_2,Dens_4,Mass_4,Vol_4,Cu_4F,...,Cu_3.1T_max,Cu_3.1T_min,FM_3.2_A,Cu_3.2C_max,Cu_3.2C_min,Ni_3.2C_max,Ni_3.2C_min,Cu_3.2T_max,Cu_3.2T_min,Ni_rec
count,30336,26213.000000,26213.000000,30336.000000,30336.000000,30336.000000,30336.000000,30336.000000,30336.000000,26083.000000,...,30335.000000,30335.000000,30335.000000,30335.000000,30335.000000,3.033500e+04,30335.0,30335.000000,30335.000000,27759.000000
mean,2024-06-06 23:52:29.999999744,2.712405,1.685584,1213.996028,1346.755948,789.375929,1.386676,811.542840,1520.387530,0.477034,...,1.627824,0.818042,0.331235,15.653601,13.036163,3.700000e+00,3.5,1.576072,0.863170,0.947898
min,2024-01-01 00:00:00,1.654600,1.141700,0.000000,-0.258293,-0.167950,0.000000,-0.083749,0.000000,0.100100,...,1.000000,0.600000,0.000000,14.000000,12.000000,3.700000e+00,3.5,1.000000,0.600000,0.594336
25%,2024-03-19 23:56:15,2.523700,1.574000,1133.500000,1310.454010,762.188248,1.378594,791.388550,1523.531250,0.396900,...,1.400000,0.600000,0.000000,15.000000,13.000000,3.700000e+00,3.5,1.400000,0.700000,0.933853
50%,2024-06-06 23:52:30,2.694400,1.685100,1274.500000,1400.961060,817.151642,1.398115,836.229218,1562.812500,0.468100,...,1.800000,0.800000,0.000000,15.500000,13.000000,3.700000e+00,3.5,1.600000,0.900000,0.943993
75%,2024-08-24 23:48:45,2.894800,1.792700,1395.500000,1475.273743,864.989029,1.415396,875.025543,1595.750000,0.541200,...,1.800000,0.900000,1.000000,16.500000,13.000000,3.700000e+00,3.5,1.800000,1.000000,0.966216
max,2024-11-11 23:45:00,4.048000,2.617300,2000.000000,2089.517578,1221.758057,1.651951,1303.474609,2028.375000,5.205700,...,2.000000,1.200000,1.000000,17.000000,15.000000,3.700000e+00,3.5,1.800000,1.300000,1.000000
std,NaN,0.284495,0.163395,304.202121,272.748762,153.755873,0.072671,147.976016,254.858332,0.176787,...,0.328567,0.170884,0.470665,0.901482,0.573167,8.881931e-16,0.0,0.236993,0.166731,0.020704


#### Отбор данных под каждую флотомашину

In [7]:
import re

In [8]:
def get_dataframe_per_fm(df, i, j):
    non_digit_features = [col for col in df.columns.tolist() if not re.findall(r'\d+', col)]
    filtered_columns_1 = [col for col in df.columns.tolist() if re.findall(f'{i}\\.{j}', col)]
    filtered_columns_2 = [col for col in df.columns.tolist() if re.findall(f'_{i}$', col)]
    return df[non_digit_features+filtered_columns_1+filtered_columns_2]

dfd = {f"fm{i}.{j}" : get_dataframe_per_fm(df, i, j) for j in range(1,3) for i in range(1,7)}

##### Чтение тестовых данных + дока

Требуется заполнить файл test.csv, указав границы диапазонов признаков (20 признаков => 40 значений границ) для выбранных моментов времени. Эти таймстемпы – 20 непрерывных временных отрезков с 15-минутной дискретностью. Каждый отрезок–тест-кейс, который будет оцениваться отдельно.

In [9]:
test_dataframe = pd.read_csv("data/test.csv")

Важно! Границы фактического диапазона в файле исходных данных – значения, которые выставляли технологи производства для сервиса оптимизации флотации (по одному на каждую линию флотомашины, например, оптимизатор для ФМ1.1.), чтобы этот оптимизатор генерировал воздействия на рычаги управления флотацией на ФМ, которые обеспечат сходимость к середине этого диапазона. Обратите внимание, что сами границы, как и ширина диапазона – плод интеллектуального труда и фантазии технологов, которые работают посменно. Технолог ведёт процесс, наблюдает за большим кол-вом параметров и не всегда уделяет достаточное внимание оптимизатору и этим диапазонам – учитывайте человеческий фактор! Наиболее релевантные значения – при включении оптимизатора (см смену значения признака с суффиксом AU с 0 на 1).
В данной задаче все границы фактических диапазонов (40 параметров типа min, max) для каждой 15-минутки ставим под сомнение и предлагаем вам сгенерировать свои варианты с учётом ограничений:
1. Каждый диапазон (признаки min, max) можно изменить не чаще 1 раза в 2 часа (не менее 8 15-минуток подряд с одной и той же парой значений границ).
2. Наименьшие допустимые приращения признаков min, max зависят от продукта, металла и ФМ (и одинаковы для двух линий одной ФМ). В качестве приращений используйте только кратные им значения (например, если наименьшее допустимое – 0.1, используйте 0.1, 0.2, 0.3, …):
    - Ni_1.*C – 0.1,
    - Cu_1.*C – 0.1,
    - Cu_2.*T – 0.01,
    - Cu_3.*T – 0.05,
    - Ni_4.*T – 0.01,
    - Ni_4.*C – 0.05,
    - Ni_5.*T – 0.01,
    - Ni_5.*C – 0.05,
    - Ni_6.*T – 0.01,
    - Ni_6.*C – 0.05.

In [10]:
# test_dataframe.isna().sum()

In [11]:
test_dataframe = test_dataframe.astype({'MEAS_DT': 'datetime64[ns]'})

In [12]:
dfd_test = {f"fm{i}.{j}" : get_dataframe_per_fm(test_dataframe, i, j) for j in range(1, 3) for i in range(1, 7)}

In [13]:
def train_test_split(train_, test_):
    train = pd.concat([train_, test_]).drop_duplicates(subset='MEAS_DT', keep=False)
    test = pd.merge(test_, train_, how='inner', on='MEAS_DT', suffixes=('_y', '_X'))
    return {
        "train" : train, 
        "test" : test
        }

In [14]:
dfd_tt = {k : train_test_split(df_train_fm, dfd_test[k]) for k, df_train_fm in dfd.items()}

In [15]:
# dfd_tt['fm1.1']['test']

feadture processing
- MEAS_DT dropping
- nan processing (catboost do it by itself) **[metric/loss-function RMSE do not allows nan value on target]**
- we have also 4 features to predict so, we number of models should have been around 6 * 2 * 4 = 48

In [16]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error
import numpy as np

In [17]:
models = {}
mae_values = {}
mape_values = {}

In [18]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [19]:
for floatmachine_name in dfd_tt.keys():

    train = dfd_tt[floatmachine_name]['train']
    test = dfd_tt[floatmachine_name]['test']

    output_features = [f.replace("_y", "") for f in test.columns.tolist() if "_y" in f]
    train = train.dropna(subset=output_features)

    for output_feature in output_features:
        print(f"training {floatmachine_name} : {output_feature}...")

        features_to_train_on = train.columns.tolist()
        features_to_train_on.remove('MEAS_DT')
        features_to_train_on = [f for f in features_to_train_on if not "min" in f and "max" not in f]

        train_pool = Pool(train[features_to_train_on], label=train[output_feature])
        test_pool = Pool(test[features_to_train_on], label=test[output_feature + "_X"])

        model = CatBoostRegressor()

        model.fit(train_pool, eval_set=test_pool, use_best_model=True, plot=False, silent=True)

        train_predictions = model.predict(train_pool)
        test_predictions = model.predict(test_pool)

        train_mae = mean_absolute_error(train[output_feature], train_predictions)
        test_mae = mean_absolute_error(test[output_feature + "_X"], test_predictions)

        train_mape = mean_absolute_percentage_error(train[output_feature], train_predictions)
        test_mape = mean_absolute_percentage_error(test[output_feature + "_X"], test_predictions)
 
        if output_feature not in mae_values:
            mae_values[output_feature] = []
            mape_values[output_feature] = []

        mae_values[output_feature].append(test_mae)
        mape_values[output_feature].append(test_mape)

        print(f"Metrics for {floatmachine_name} : {output_feature}")
        print(f"Train MAE: {round(train_mae, 4)}, Train MAPE: {round(train_mape, 4)}%")
        print(f"Test MAE: {round(test_mae, 4)}, Test MAPE: {round(test_mape, 4)}%")
        print("----------------------------------//----------------------------------")

        models[f"{floatmachine_name}:{output_feature}"] = model

training fm1.1 : Ni_1.1C_min...
Metrics for fm1.1 : Ni_1.1C_min
Train MAE: 0.0468, Train MAPE: 1.7921%
Test MAE: 0.0435, Test MAPE: 1.6678%
----------------------------------//----------------------------------
training fm1.1 : Ni_1.1C_max...
Metrics for fm1.1 : Ni_1.1C_max
Train MAE: 0.0626, Train MAPE: 1.9974%
Test MAE: 0.0943, Test MAPE: 3.0664%
----------------------------------//----------------------------------
training fm1.1 : Cu_1.1C_min...
Metrics for fm1.1 : Cu_1.1C_min
Train MAE: 0.0815, Train MAPE: 1.8806%
Test MAE: 0.1097, Test MAPE: 2.5794%
----------------------------------//----------------------------------
training fm1.1 : Cu_1.1C_max...
Metrics for fm1.1 : Cu_1.1C_max
Train MAE: 0.0441, Train MAPE: 0.908%
Test MAE: 0.042, Test MAPE: 0.8639%
----------------------------------//----------------------------------
training fm2.1 : Cu_2.1T_min...
Metrics for fm2.1 : Cu_2.1T_min
Train MAE: 0.007, Train MAPE: 2.4123%
Test MAE: 0.0192, Test MAPE: 6.6704%
-------------------

In [ ]:
# for output_feature in mae_values.keys():
#     mean_mae = sum(mae_values[output_feature]) / len(mae_values[output_feature])
#     mean_mape = sum(mape_values[output_feature]) / len(mape_values[output_feature])
#     print(f"Mean MAE for {output_feature}: {round(mean_mae, 4)}")
#     print(f"Mean MAPE for {output_feature}: {round(mean_mape, 4)}%")

In [29]:
mae_list = [v[0] for _,v in mae_values.items()]
mape_list = [v[0] for _,v in mape_values.items()]

print(f"Mean MAE {round(sum(mae_list) / len(mae_list), 4)}")
print(f"Mean MAPE {round(sum(mape_list) / len(mape_list), 4)}%")

Mean MAE 0.0674
Mean MAPE 3.3357%


In [30]:
models = {}
mae_values = {}
mape_values = {}

In [31]:
for floatmachine_name in dfd_tt.keys():

    train = dfd_tt[floatmachine_name]['train']
    test = dfd_tt[floatmachine_name]['test']

    output_features = [f.replace("_y", "") for f in test.columns.tolist() if "_y" in f]
    train = train.dropna(subset=output_features)

    for output_feature in output_features:
        print(f"training {floatmachine_name} : {output_feature}...")

        features_to_train_on = train.columns.tolist()
        features_to_train_on.remove('MEAS_DT')
        features_to_train_on = [f for f in features_to_train_on if not "min" in f and "max" not in f and "FM_" not in f]

        train_pool = Pool(train[features_to_train_on], label=train[output_feature])
        test_pool = Pool(test[features_to_train_on], label=test[output_feature + "_X"])

        model = CatBoostRegressor()

        model.fit(train_pool, eval_set=test_pool, use_best_model=True, plot=False, silent=True)

        train_predictions = model.predict(train_pool)
        test_predictions = model.predict(test_pool)

        train_mae = mean_absolute_error(train[output_feature], train_predictions)
        test_mae = mean_absolute_error(test[output_feature + "_X"], test_predictions)

        train_mape = mean_absolute_percentage_error(train[output_feature], train_predictions)
        test_mape = mean_absolute_percentage_error(test[output_feature + "_X"], test_predictions)
 
        if output_feature not in mae_values:
            mae_values[output_feature] = []
            mape_values[output_feature] = []

        mae_values[output_feature].append(test_mae)
        mape_values[output_feature].append(test_mape)

        # print(f"Metrics for {floatmachine_name} : {output_feature}")
        # print(f"Train MAE: {round(train_mae, 4)}, Train MAPE: {round(train_mape, 4)}%")
        # print(f"Test MAE: {round(test_mae, 4)}, Test MAPE: {round(test_mape, 4)}%")
        # print("----------------------------------//----------------------------------")

        models[f"{floatmachine_name}:{output_feature}"] = model

training fm1.1 : Ni_1.1C_min...
training fm1.1 : Ni_1.1C_max...
training fm1.1 : Cu_1.1C_min...
training fm1.1 : Cu_1.1C_max...
training fm2.1 : Cu_2.1T_min...
training fm2.1 : Cu_2.1T_max...
training fm3.1 : Cu_3.1T_min...
training fm3.1 : Cu_3.1T_max...
training fm4.1 : Ni_4.1T_min...
training fm4.1 : Ni_4.1T_max...
training fm4.1 : Ni_4.1C_min...
training fm4.1 : Ni_4.1C_max...
training fm5.1 : Ni_5.1T_min...
training fm5.1 : Ni_5.1T_max...
training fm5.1 : Ni_5.1C_min...
training fm5.1 : Ni_5.1C_max...
training fm6.1 : Ni_6.1T_min...
training fm6.1 : Ni_6.1T_max...
training fm6.1 : Ni_6.1C_min...
training fm6.1 : Ni_6.1C_max...
training fm1.2 : Ni_1.2C_min...
training fm1.2 : Ni_1.2C_max...
training fm1.2 : Cu_1.2C_min...
training fm1.2 : Cu_1.2C_max...
training fm2.2 : Cu_2.2T_min...
training fm2.2 : Cu_2.2T_max...
training fm3.2 : Cu_3.2T_min...
training fm3.2 : Cu_3.2T_max...
training fm4.2 : Ni_4.2T_min...
training fm4.2 : Ni_4.2T_max...
training fm4.2 : Ni_4.2C_min...
training

In [ ]:
# Mean MAE 0.0674
# Mean MAPE 3.3357%

In [32]:
mae_list = [v[0] for _,v in mae_values.items()]
mape_list = [v[0] for _,v in mape_values.items()]

print(f"Mean MAE {round(sum(mae_list) / len(mae_list), 4)}")
print(f"Mean MAPE {round(sum(mape_list) / len(mape_list), 4)}%")

Mean MAE 0.0705
Mean MAPE 3.5266%
